In [ ]:
"""
CODEBASICS INSPIRED!
"""

In [11]:
import requests
import os
import streamlit as st
import pickle
import time
import langchain
from langchain import OpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import json
import subprocess

In [25]:
import requests
from bs4 import BeautifulSoup
from langchain.schema import Document

url = "https://python.langchain.com/docs/how_to/tools_chain/"
res = requests.get(url)
soup = BeautifulSoup(res.text, "html.parser")

# Extract main content
text = soup.get_text()

# Wrap in LangChain Document
doc = Document(page_content=text, metadata={"source": url})
print(doc.page_content[:500])  # preview






How to use tools in a chain | 🦜️🔗 LangChain






Skip to main contentWe are growing and hiring for multiple roles for LangChain, LangGraph and LangSmith.  Join our team!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Aug


In [27]:
from langchain.llms import Ollama
llm = Ollama(model="gemma3:4b")

/var/folders/hv/3fk9wmx12gs74mp51cnpmhw80000gq/T/ipykernel_90781/4116184385.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="gemma3:4b")


In [29]:
from langchain_core.prompts import PromptTemplate

In [35]:
template = '''You are a API documentation interpretation expert, I will be giving you an unstructured scraped API documentation of langchain in python language
You will go through the entire documentation, The API wil have multiple code blocks and reasoning about what they do
You are supposed to return the documentation in a structured format, with each code block and the respective reasoning in a jsonL format
where "question" will have the reasoning as to what the code block does
and "answer" will have the code block
    {api_doc}
    '''
pt = PromptTemplate.from_template(template)
chain = pt | llm
response = chain.invoke(input={"api_doc": doc})

In [36]:
print(response)

```jsonl
[
  {
    "question": "This document outlines how to use tools within LangChain, primarily focusing on creating chains and agents that call tools. Tools can be anything from APIs to functions to databases. The core idea is to extend the capabilities of an LLM beyond just text output by strategically utilizing these external resources. Proper prompting and response parsing are key to ensuring the model selects the right tools and provides the necessary inputs for them. It also lays out the setup and installation steps including installing LangChain and any required dependencies.",
    "answer": "We'll need to install the following packages for this guide:\n%pip install --upgrade --quiet langchain\nIf you'd like to trace your runs in LangSmith uncomment and set the following environment variables:\nimport getpassimport os\n# os.environ[\"LANGSMITH_TRACING\"] = \"true\"\n# os.environ[\"LANGSMITH_API_KEY\"] = getpass.getpass(\"Enter API key for OpenAI:\")"
  },
  {
    "question":

In [2]:
BASE_URL = "https://python.langchain.com"
GUIDES_URL = f"{BASE_URL}/docs/how_to/"

In [3]:
# Step 1: Scrape all guide URLs
def get_guide_links():
    response = requests.get(GUIDES_URL)
    soup = BeautifulSoup(response.content, "html.parser")
    links = []

    for a in soup.find_all("a", href=True):
        if a.text.strip().lower().startswith("how to"):
            full_url = urljoin(BASE_URL, a['href'])
            links.append((a.text.strip(), full_url))

    return links

In [8]:
# Step 2: Scrape one guide and extract title, content sections, code
def scrape_guide(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    print(soup)

    # title = soup.find("h1").text.strip() if soup.find("h1") else "Untitled"
    # sections = []
    # current_heading = None
    # current_content = []
    # current_code_blocks = []

    # for tag in soup.find_all(["h2", "p", "pre"]):
    #     if tag.name == "h2":
    #         if current_heading:
    #             sections.append({
    #                 "heading": current_heading,
    #                 "text": " ".join(current_content).strip(),
    #                 "code_blocks": current_code_blocks
    #             })
    #         current_heading = tag.text.strip()
    #         current_content = []
    #         current_code_blocks = []
    #     elif tag.name == "p":
    #         current_content.append(tag.text.strip())
    #     elif tag.name == "pre":
    #         current_code_blocks.append(tag.get_text("\n", strip=False))

    # # Add the final section
    # if current_heading:
    #     # Use DeepSeek LLM via OLLAMA to convert section into Q&A format

    #     section_text = " ".join(current_content).strip()
    #     code_blocks_text = "\n\n".join(current_code_blocks)
    #     prompt = f"""You are an expert in LangChain. Given the following section from the documentation, generate a question and a detailed answer that would help someone learn LangChain. Include code examples if present.

    # Section Heading: {current_heading}
    # Section Text: {section_text}
    # Code Blocks:
    # {code_blocks_text}

    # Format your response as:
    # Question: <question>
    # Answer: <answer>
    # """

    #     # Call OLLAMA DeepSeek LLM locally
    #     ollama_cmd = [
    #         "ollama", "run", "gemma3:4b",
    #         "--prompt", prompt
    #     ]
    #     try:
    #         result = subprocess.run(
    #         ollama_cmd,
    #         capture_output=True,
    #         text=True,
    #         timeout=60
    #         )
    #         output = result.stdout.strip()
    #         # Parse output into question and answer
    #         if "Question:" in output and "Answer:" in output:
    #             q = output.split("Question:", 1)[1].split("Answer:", 1)[0].strip()
    #             a = output.split("Answer:", 1)[1].strip()
    #         else:
    #             q = f"What is covered in the section '{current_heading}'?"
    #             a = section_text + ("\n\n" + code_blocks_text if code_blocks_text else "")
    #     except Exception as e:
    #         q = f"What is covered in the section '{current_heading}'?"
    #         a = section_text + ("\n\n" + code_blocks_text if code_blocks_text else "")

    #     sections.append({
    #         "heading": current_heading,
    #         "question": q,
    #         "answer": a
    #     })

    # return {
    #     "title": title,
    #     "url": url,
    #     "sections": sections
    # }

In [5]:
all_links = get_guide_links()

In [6]:
all_links[0:5]

[('How to use tools in a chain',
  'https://python.langchain.com/docs/how_to/tools_chain/'),
 ('How to use a vectorstore as a retriever',
  'https://python.langchain.com/docs/how_to/vectorstore_retriever/'),
 ('How to add memory to chatbots',
  'https://python.langchain.com/docs/how_to/chatbots_memory/'),
 ('How to use example selectors',
  'https://python.langchain.com/docs/how_to/example_selectors/'),
 ('How to add a semantic layer over graph database',
  'https://python.langchain.com/docs/how_to/graph_semantic/')]

In [7]:
temp_link = all_links[0][1]  # Just take the first link for testing
temp_link

'https://python.langchain.com/docs/how_to/tools_chain/'

In [9]:
data = scrape_guide(temp_link)

<!DOCTYPE html>

<html class="docs-wrapper plugin-docs plugin-id-default docs-version-current docs-doc-page docs-doc-id-how_to/tools_chain" data-has-hydrated="false" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="Docusaurus v3.5.2" name="generator"/>
<title data-rh="true">How to use tools in a chain | 🦜️🔗 LangChain</title><meta content="width=device-width,initial-scale=1" data-rh="true" name="viewport"/><meta content="summary_large_image" data-rh="true" name="twitter:card"/><meta content="https://python.langchain.com/img/brand/theme-image.png" data-rh="true" property="og:image"/><meta content="https://python.langchain.com/img/brand/theme-image.png" data-rh="true" name="twitter:image"/><meta content="https://python.langchain.com/docs/how_to/tools_chain/" data-rh="true" property="og:url"/><meta content="en" data-rh="true" property="og:locale"/><meta content="en" data-rh="true" name="docusaurus_locale"/><meta content="en" data-rh="true" name="docsearch:language"/><meta

In [17]:
data

{'title': 'How to use tools in a chain',
 'url': 'https://python.langchain.com/docs/how_to/tools_chain/',
 'sections': [{'heading': 'Setup\u200b',
   'text': "We'll need to install the following packages for this guide: If you'd like to trace your runs in LangSmith uncomment and set the following environment variables:",
   'code_blocks': ['%\npip install \n-\n-\nupgrade \n-\n-\nquiet langchain',
    'import\n getpass\nimport\n os\n# os.environ["LANGSMITH_TRACING"] = "true"\n# os.environ["LANGSMITH_API_KEY"] = getpass.getpass()']},
  {'heading': 'Create a tool\u200b',
   'text': 'First, we need to create a tool to call. For this example, we will create a custom tool from a function. For more information on creating custom tools, please see this guide.',
   'code_blocks': ['from\n langchain_core\n.\ntools \nimport\n tool\n@tool\ndef\n \nmultiply\n(\nfirst_int\n:\n \nint\n,\n second_int\n:\n \nint\n)\n \n-\n>\n \nint\n:\n    \n"""Multiply two integers together."""\n    \nreturn\n first_i

In [19]:
print(data["sections"][-2]["code_blocks"])

['pip install -qU "langchain[openai]"', 'import\n getpass\nimport\n os\nif\n \nnot\n os\n.\nenviron\n.\nget\n(\n"OPENAI_API_KEY"\n)\n:\n  os\n.\nenviron\n[\n"OPENAI_API_KEY"\n]\n \n=\n getpass\n.\ngetpass\n(\n"Enter API key for OpenAI: "\n)\nfrom\n langchain\n.\nchat_models \nimport\n init_chat_model\nllm \n=\n init_chat_model\n(\n"gpt-4o-mini"\n,\n model_provider\n=\n"openai"\n)', 'llm_with_tools \n=\n llm\n.\nbind_tools\n(\n[\nmultiply\n]\n)', 'msg \n=\n llm_with_tools\n.\ninvoke\n(\n"whats 5 times forty two"\n)\nmsg\n.\ntool_calls', "[{'name': 'multiply',\n  'args': {'first_int': 5, 'second_int': 42},\n  'id': 'call_8QIg4QVFVAEeC1orWAgB2036',\n  'type': 'tool_call'}]", 'from\n operator \nimport\n itemgetter\nchain \n=\n llm_with_tools \n|\n \n(\nlambda\n x\n:\n x\n.\ntool_calls\n[\n0\n]\n[\n"args"\n]\n)\n \n|\n multiply\nchain\n.\ninvoke\n(\n"What\'s four times 23"\n)', '92']


In [7]:
# Step 3: Run full scrape and save JSONL for fine-tuning
def scrape_all_guides_and_save(filename="langchain_howto.jsonl"):
    all_links = get_guide_links()
    with open(filename, "w", encoding="utf-8") as f:
        for name, url in all_links:
            print(f"Scraping: {name}")
            data = scrape_guide(url)
            f.write(json.dumps(data, ensure_ascii=False) + "\n")

scrape_all_guides_and_save()


Scraping: How to use tools in a chain
Scraping: How to use a vectorstore as a retriever
Scraping: How to add memory to chatbots
Scraping: How to use example selectors
Scraping: How to add a semantic layer over graph database
Scraping: How to invoke runnables in parallel
Scraping: How to stream chat model responses
Scraping: How to add default invocation args to a Runnable


KeyboardInterrupt: 